In [97]:
from sklearn import datasets, model_selection,metrics
import pandas as pd
import numpy as np

In [2]:
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = pd.Series(iris.target)

In [3]:
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
y = df_iris['target'].values
df_iris.pop('target')
X = df_iris.values

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [6]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [7]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [8]:
def mse(y_pred, y_true):
    return ((y_true - y_pred) ** 2).mean()

In [167]:
class Perceptron:
    def __init__(self, n_inputs, n_outputs, n_layers, n_neurons, activation=None, max_iter=1000, eta=0.1):
        # количество входящих нейронов
        self.n_inputs = n_inputs
        # количество скрытых слоев
        self.n_outputs = n_outputs
        self.n_layers = n_layers
        # количество нейронов на скрытом слое
        self.n_neurons = n_neurons
        # создание скрытых слоев
        self.layers = []
        self.activation = activation
        self.max_iter = max_iter
        self.eta = eta
        
    def create_layers(self):
        n_inputs = self.n_inputs
        # создание скрытых слоев
        for i in range(self.n_layers):
            self.layers.append(self.NeuronLayer(n_inputs, self.n_neurons))
            n_inputs = self.n_neurons
        # создание выходного слоя
        self.layers.append(self.NeuronLayer(n_inputs, self.n_outputs))
        
    def back_propogation(self, error, values, X):
        weights = 1
        for i in reversed(range(len(self.layers))):
            self.layers[i].delta = np.dot(error, np.transpose(weights))
            derivative = np.transpose(sigmoid_derivative(values))
            for j in range(self.layers[i].delta.shape[1]):
                self.layers[i].delta[0][j] *= derivative[0][j]
            if i == 0:
                values = X
            else:
                values = self.layers[i - 1].values
            values = values.reshape(-1, 1)
            self.layers[i].weights -= self.eta * np.dot(values, self.layers[i].delta)
            weights = self.layers[i].weights
            error = self.layers[i].delta
            
    def get_weights(self):
        weights = []
        for layer in self.layers:
            weights.append(layer.weights)
        return weights
            
    def forward_propogation(self, x):
        inputs = x
        for layer in self.layers:
            if self.activation is not None:
                layer.values = self.activation(np.dot(inputs, layer.weights) + layer.b)
            else:
                layer.values = np.dot(inputs, layer.weights) + layer.b
            inputs = layer.values
        return inputs
    
    def fit(self, X, y):
        self.create_layers()
        for i in range(self.max_iter):
            index = np.random.randint(0, X.shape[0])
            inputs = X[index].reshape(1, -1)
            outputs = self.forward_propogation(inputs)
            error = outputs - y[index]
            self.back_propogation(error, outputs, X[index])
            
    def predict(self, X):
        outputs = np.ones((X.shape[0], 1))
        for i in range(X.shape[0]):
            outputs[i] = self.forward_propogation(X[i].reshape(1, -1))
        return outputs
            
    class NeuronLayer:
        def __init__(self, n_inputs, n_neurons):
            self.weights = np.random.normal(loc=0, scale=0.1, size=(n_inputs, n_neurons))
            self.values = []
            self.delta = []
            self.b = np.ones((1, n_neurons))

In [168]:
perceptron = Perceptron(4, 1, 2, 2, sigmoid)
perceptron.fit(X_train, y_train)

In [164]:
y_pred = perceptron.predict(X_test)